In [1]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)

In [2]:
#numpy
import numpy as np

#torch
import torch, torch.nn as nn
import torch.nn.functional as F


In [3]:
from xml.dom import minidom
import re

def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

def make_a_search_request(query):
    # return json
    # json will be processed further
    params = {
            "apikey": "0833a307-97d3-462a-99d9-27db400c70da",
            "query": query,
            "index": ["cw12"],
            "size": 1000,
            "pretty": True
        }
    response = requests.get(url = "http://www.chatnoir.eu/api/v1/_search", params = params)
    return response

def clean_punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()


** load bert model **

In [4]:
import torch

from pytorch_transformers.modeling_bert import BertConfig, BertForPreTraining, load_tf_weights_in_bert


tf_checkpoint_path="/notebook/uncased_L-12_H-768_A-12/bert_model.ckpt"
bert_config_file = "/notebook/uncased_L-12_H-768_A-12/bert_config.json"
pytorch_dump_path="/notebook/uncased_L-12_H-768_A-12/pytorch_model"

config = BertConfig.from_json_file(bert_config_file)
print("Building PyTorch model from configuration: {}".format(str(config)))
model = BertForPreTraining(config)

# Load weights from tf checkpoint
load_tf_weights_in_bert(model, config, tf_checkpoint_path)

# Save pytorch-model
print("Save PyTorch model to {}".format(pytorch_dump_path))
torch.save(model.state_dict(), pytorch_dump_path)

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/tensorflow/pyth

Save PyTorch model to /notebook/uncased_L-12_H-768_A-12/pytorch_model


In [5]:
import pytorch_transformers
from pytorch_transformers import *

config_class, model_class, tokenizer_class = BertConfig, BertModel, BertTokenizer

config = config_class.from_pretrained('bert-base-uncased')
config.output_attentions=True
model = model_class.from_pretrained('bert-base-uncased', config=config)


In [6]:
response = make_a_search_request("What is difference between sex and love?")

In [7]:
output_dir = '/notebook/touche/output/'
input_dir = '/notebook/touche/'
input_file = 'topics-task-2-only-titles.xml'
list_of_tuples = read_xml(input_dir + input_file)
common_list = []
 
for elem in list_of_tuples[:5]:
    qid = elem[0]
    Q0 = 'Q0'
    query = elem[1]
    tag = 'MethodAttention'
    response = make_a_search_request(query)
    try:
        getted_request = response.json()
    except:
        print ("exept0")
        #return getted_request

    n_request = 100
    while (n_request > 0 and 'results' not in getted_request):
        response = make_a_search_request(query)
        getted_request = response.json()
        n_request == n_request - 1
        print ("rerequest", query)

    try:
        scores0 = [elem['score'] for elem in getted_request['results']]
        #print ("0")
        docs = [elem['trec_id'] for elem in getted_request['results']]
        #print ("1")
        titles = [elem['title'] for elem in getted_request['results']]
        #print ("2")
        answers_bodies = [cleanhtml(elem['snippet']) for elem in getted_request['results']]
        #print ("3")
        # print (scores0, scores3, scores)
    except:
        print ("exept1")
        #return getted_request

1
[<DOM Element: topics at 0x7fe9c4dcf230>]
50


In [15]:
def run_baseline(output_dir = '/notebook/touche/output/', input_dir = '/notebook/touche/', input_file = 'topics-task-2-only-titles.xml'):
    list_of_tuples = read_xml(input_dir + input_file)
    common_list = []
        
    print ("attention")    
    for elem in list_of_tuples[:5]:
        qid = elem[0]
        Q0 = 'Q0'
        query = elem[1]
        tag = 'MethodAttention'
        response = make_a_search_request(query)
        try:
            getted_request = response.json()
        except:
            print ("exept0")
            #return getted_request

        n_request = 100
        while (n_request > 0 and 'results' not in getted_request):
            response = make_a_search_request(query)
            getted_request = response.json()
            n_request == n_request - 1
            print ("rerequest", query)

        try:
            scores0 = [elem['score'] for elem in getted_request['results']]
            #print ("0")
            docs = [elem['trec_id'] for elem in getted_request['results']]
            #print ("1")
            titles = [elem['title'] for elem in getted_request['results']]
            #print ("2")
            answers_bodies = [cleanhtml(elem['snippet']) for elem in getted_request['results']]
            #print ("3")
            # print (scores0, scores3, scores)
        except:
            print ("exept1")
            #return getted_request
        scores = make_scores_transformers(query, answers_bodies)
        qids = qid*len(scores)
        Q0s = [Q0 for elem in scores]
        queries = query*len(scores)
        tags = [tag for elem in scores]
        part_of_commom_list = list(zip(qids, Q0s, docs, scores, tags))
        part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

        qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

        ranks = range(1, len(scores) + 1)
        part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
        common_list = common_list + part_of_commom_list

    with open(output_dir + 'run_example1.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [16]:
output_dir = '/notebook/touche/output/'
input_dir = '/notebook/touche/'
input_file = 'topics-task-2-only-titles.xml'
list_of_tuples = read_xml(input_dir + input_file)
common_list = []

print (list_of_tuples)
print ("lstm")    
for elem in list_of_tuples[:5]:
    qid = elem[0]
    Q0 = 'Q0'
    query = elem[1]
    tag = 'MethodAttention'
    response = make_a_search_request(query)
    try:
        getted_request = response.json()
    except:
        print ("exept0")
        #return getted_request

    n_request = 100
    while (n_request > 0 and 'results' not in getted_request):
        response = make_a_search_request(query)
        getted_request = response.json()
        n_request == n_request - 1
        print ("rerequest", query)

    try:
        scores0 = [elem['score'] for elem in getted_request['results']]
        #print ("0")
        docs = [elem['trec_id'] for elem in getted_request['results']]
        #print ("1")
        titles = [elem['title'] for elem in getted_request['results']]
        #print ("2")
        answers_bodies = [cleanhtml(elem['snippet']) for elem in getted_request['results']]
        #print ("3")
        # print (scores0, scores3, scores)
    except:
        print ("exept1")
        #return getted_request
    scores = make_scores_transformers(query, answers_bodies)
    qids = qid*len(scores)
    Q0s = [Q0 for elem in scores]
    queries = query*len(scores)
    tags = [tag for elem in scores]
    part_of_commom_list = list(zip(qids, Q0s, docs, scores, tags))
    part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 

    qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)

    ranks = range(1, len(scores) + 1)
    part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
    common_list = common_list + part_of_commom_list

with open(output_dir + 'run_example4.txt', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

1
[<DOM Element: topics at 0x7f6750a8c690>]
50
[('1', 'What is the difference between sex and love?'), ('2', 'Which is better, laptop or desktop?'), ('3', 'Which is better, Canon or Nikon?'), ('4', 'What are the best dish detergents?'), ('5', 'What are the best cities to live?'), ('6', 'What is the longest river in the U.S.?'), ('7', 'Which is healthiest: coffee, green tea or black tea and why?'), ('8', 'What are advantages and disadvantages of PHP over Python and vice versa?'), ('9', 'Why is Linux better than Windows?'), ('10', 'How to sleep better?'), ('11', 'Should I buy an LCD TV or a plasma TV?'), ('12', 'Train or plane? Which is the better choice?'), ('13', 'What is the highest mountain on Earth?'), ('14', 'Should one prefer Chinese medicine or Western medicine?'), ('15', 'What are the best washing machine brands?'), ('16', 'Should I buy or rent?'), ('17', 'Do you prefer cats or dogs, and why?'), ('18', 'What is the better way to grill outdoors: gas or charcoal?'), ('19', 'Which 

NameError: name 'make_scores_transformers' is not defined

In [ ]:
def make_scores_transformers(query, answers_bodies):
    scores = []
    for ind, elem in enumerate(answers_bodies):
        sentences = ["[CLS] " + query + " [SEP] " + answers_bodies[ind] + " [SEP]"]
        #print(sentences[0])

        # Tokenize with BERT tokenizer
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
        #print ("Tokenize the first sentence:")
        #print (tokenized_texts[0])
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
        # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
        segments_ids = (len(tokenizer.tokenize(query)) + 2)*[0] + (len(tokenizer.tokenize(answers_bodies[ind])) + 1)*[1]
        cls_idx = 0
        sep_idx = len(tokenizer.tokenize(query))
        end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answers_bodies[ind])) + 3
        #print (len(tokenizer.tokenize(query)), len(tokenizer.tokenize(answers_bodies[ind])))
        #print (len(indexed_tokens), len(segments_ids))
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        answ = model(tokens_tensor, segments_tensors)

        attensions = answ[2][3][0][0].detach().numpy() + answ[2][9][0][0].detach().numpy()
        #probs[cls_idx:sep_idx, sep_idx:] = 0.0
        score = np.sum(attensions[cls_idx:sep_idx, sep_idx:]) + np.sum(attensions[sep_idx:end_idx, cls_idx:sep_idx+2])
        #print (score)
        scores.append(score)
    return scores
    

In [ ]:
sentences = ["[CLS] " + query + " [SEP] " + answers_bodies[10] + " [SEP]"]
print(sentences[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = (len(tokenizer.tokenize(query)) + 2)*[0] + (len(tokenizer.tokenize(answers_bodies[10])) + 1)*[1]
print (len(tokenizer.tokenize(query)), len(tokenizer.tokenize(answers_bodies[10])))
print (len(indexed_tokens), len(segments_ids))
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)

attensions = answ[2][3][0][0].detach().numpy() + answ[2][9][0][0].detach().numpy()

In [ ]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)

In [ ]:
answ[0].shape

In [ ]:
answ[1].shape

In [ ]:
answ[2][3].shape

In [22]:
import bokeh.plotting as pl
import bokeh.models as bm
from bokeh.io import output_notebook, show
output_notebook()

def draw_attention(inp_tokens, trans_tokens, probs):
    """ An intentionally ambiguous function to visualize attention weights """
    probs = probs[:len(trans_tokens), :len(inp_tokens)]
    
    fig = pl.figure(x_range=(0, len(inp_tokens)), y_range=(0, len(trans_tokens)),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs[::-1]], 0, 0, len(inp_tokens), len(trans_tokens))

    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens)) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens)) + 0.5, inp_tokens))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens)) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens)) + 0.5, trans_tokens[::-1]))

    show(fig)

Loading BokehJS ...

In [23]:
np.arange(len(inp_tokens)) + 0.5

array([0.5])

Идея - из аттеншн слоев выбрать те, которые в левом вернем и правом нижнем углу (0 предложение на себя и  предложение на себя) не дают полностью черных квадратов. 3 слой - атеншн на аналогичные слова

обнулить sep-tokena

Просуммировать левый верхний и правый нижний квадрат - для каждого поискового ответа

In [24]:
probs = answ[2][ii][0][0].detach().numpy()

probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]

NameError: name 'ii' is not defined

In [25]:
probs

array([[3.04901358e-02, 1.73009280e-02, 2.71671321e-02, 3.23435036e-03,
        7.38640316e-03, 6.50974363e-03, 1.75563805e-02, 2.83113308e-03,
        5.73212095e-03, 1.99074615e-02, 4.25227046e-01, 2.68580541e-02,
        4.91099199e-03, 6.71137124e-03, 1.78290950e-03, 3.86589556e-03,
        1.37922494e-03, 1.97049649e-03, 3.40984343e-03, 3.30288596e-02,
        1.13582850e-01, 9.59249493e-03, 5.67750400e-03, 4.01255675e-03,
        2.00413652e-02, 1.99832708e-01],
       [4.95518185e-02, 7.54447058e-02, 3.23766805e-02, 7.63435801e-03,
        2.47847065e-02, 2.84025772e-03, 1.26477480e-02, 5.21405134e-04,
        5.71716484e-03, 1.92897245e-02, 3.86078000e-01, 2.62657180e-02,
        7.93578476e-03, 6.01078989e-03, 3.45585961e-03, 1.38545467e-03,
        6.03198248e-04, 3.27736069e-03, 1.69685995e-03, 3.38014751e-03,
        1.15208216e-02, 1.03616284e-03, 3.82963172e-03, 1.07755396e-03,
        6.31348370e-03, 3.05324614e-01],
       [1.44388424e-02, 2.57208198e-02, 5.70744723e-02

In [26]:
cls_idx = 0
sep_idx = len(tokenizer.tokenize(query))
end_idx = len(tokenizer.tokenize(query)) + len(tokenizer.tokenize(answers_bodies[10])) + 3

In [27]:
probs = answ[2][ii][0][0].detach().numpy()
probs = probs[cls_idx:sep_idx, sep_idx:end_idx]
probs.shape

NameError: name 'ii' is not defined

In [28]:
len(trans_tokens[0][cls_idx:sep_idx])

9

In [35]:
!pip3 install selenium

     |████████████████████████████████| 911kB 810kB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [36]:
from bokeh.io import export_png

In [38]:
trans_tokens = tokenized_texts
inp_tokens = tokenized_texts

for ii in range(12):
    probs = answ[2][ii][0][0].detach().numpy()

    #probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
    probs = probs
    #probs[cls_idx:sep_idx, sep_idx:] = 0.0
    #probs[sep_idx:end_idx, cls_idx:sep_idx+2] = 0.0
    
    #nullify sep - need we or not?
    #probs1 = np.copy(probs)
    #probs1[0] = 0.0
    #probs1[:, 0] = 0.0
    #probs1[10] = 0.0
    #probs1[len(probs)-1] = 0.0
    #probs1[:, 10] = 0.0
    #probs1[:, len(probs)-1] = 0.0

    fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))

    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))
    #export_png(fig, filename="plot.png")
    show(fig)
    

In [ ]:
trans_tokens = tokenized_texts
inp_tokens = tokenized_texts

for ii in [3, 9]:
    probs = answ[2][ii][0][0].detach().numpy()

    #probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
    probs = probs[cls_idx:sep_idx, :]
    
    #nullify sep - need we or not?
    #probs1 = np.copy(probs)
    #probs1[0] = 0.0
    #probs1[:, 0] = 0.0
    #probs1[10] = 0.0
    #probs1[len(probs)-1] = 0.0
    #probs1[:, 10] = 0.0
    #probs1[:, len(probs)-1] = 0.0

    fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
    fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))

    fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
    fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
    fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
    fig.xaxis.major_label_orientation = 45

    fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
    fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
    fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

    show(fig)

In [17]:
sentences = ["[CLS] " + query + " [SEP] " + query + " [SEP]"]
print(sentences[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] + 10*[1]

[CLS] What is the difference between sex and love? [SEP] What is the difference between sex and love? [SEP]
Tokenize the first sentence:
['[CLS]', 'what', 'is', 'the', 'difference', 'between', 'sex', 'and', 'love', '?', '[SEP]', 'what', 'is', 'the', 'difference', 'between', 'sex', 'and', 'love', '?', '[SEP]']


In [18]:
len(segments_ids)

21

In [19]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)

In [20]:
trans_tokens = tokenized_texts
inp_tokens = tokenized_texts

probs = answ[2][3][0][0].detach().numpy()
probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))

fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
fig.xaxis.major_label_orientation = 45

fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

show(fig)

NameError: name 'pl' is not defined

In [21]:
sentences = ["[CLS] " + query + " [SEP] " + 'Just the place for a Snark!" the Bellman cried' + " [SEP]"]
print(sentences[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_texts[0])
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] + 15*[1]

tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
answ = model(tokens_tensor, segments_tensors)

trans_tokens = tokenized_texts
inp_tokens = tokenized_texts

probs = answ[2][9][0][0].detach().numpy()
probs = probs[:len(trans_tokens[0]), :len(inp_tokens[0])]
fig = pl.figure(x_range=(0, len(inp_tokens[0])), y_range=(0, len(trans_tokens[0])),
                    x_axis_type=None, y_axis_type=None, tools=[])
fig.image([probs], 0, 0, len(inp_tokens[0]), len(trans_tokens[0]))

fig.add_layout(bm.LinearAxis(axis_label='source tokens'), 'above')
fig.xaxis.ticker = np.arange(len(inp_tokens[0])) + 0.5
fig.xaxis.major_label_overrides = dict(zip(np.arange(len(inp_tokens[0])) + 0.5, inp_tokens[0]))
fig.xaxis.major_label_orientation = 45

fig.add_layout(bm.LinearAxis(axis_label='translation tokens'), 'left')
fig.yaxis.ticker = np.arange(len(trans_tokens[0])) + 0.5
fig.yaxis.major_label_overrides = dict(zip(np.arange(len(trans_tokens[0])) + 0.5, trans_tokens[0]))

show(fig)

[CLS] What is the difference between sex and love? [SEP] Just the place for a Snark!" the Bellman cried [SEP]
Tokenize the first sentence:
['[CLS]', 'what', 'is', 'the', 'difference', 'between', 'sex', 'and', 'love', '?', '[SEP]', 'just', 'the', 'place', 'for', 'a', 's', '##nar', '##k', '!', '"', 'the', 'bell', '##man', 'cried', '[SEP]']


NameError: name 'pl' is not defined

In [17]:
def run_baseline(output_dir = '/notebook/touche/output/', input_dir = '/notebook/touche/', input_file = 'topics-task-2-only-titles.xml'):
    list_of_tuples = read_xml(input_dir + input_file)
    common_list = []
    
    with open(output_dir + 'run_example.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
        
    print ("lstm")    
    for elem in list_of_tuples:
        qid = elem[0]
        Q0 = 'Q0'
        query = elem[1]
        tag = 'Method3'
        response = make_a_search_request(query)
        try:
            getted_request = response.json()
        except:
            print ("exept0")
            return getted_request
        
        n_request = 100
        while (n_request > 0 and 'results' not in getted_request):
            response = make_a_search_request(query)
            getted_request = response.json()
            n_request == n_request - 1
            print ("rerequest", query)
            
        try:
            scores0 = [elem['score'] for elem in getted_request['results']]
            print ("0")
            docs = [elem['trec_id'] for elem in getted_request['results']]
            print ("1")
            titles = [elem['title'] for elem in getted_request['results']]
            print ("2")
            answers_bodies = [cleanhtml(elem['snippet']) for elem in getted_request['results']]
            print ("3")
            # print (scores0, scores3, scores)
        except:
            print ("exept1")
            return getted_request
        scores = make_scores_lstm(query, answers_bodies)
        qids = qid*len(scores)
        Q0s = [Q0 for elem in scores]
        queries = query*len(scores)
        tags = [tag for elem in scores]
        part_of_commom_list = list(zip(qids, Q0s, docs, scores, tags))
        part_of_commom_list = sorted(part_of_commom_list, key = lambda x: x[3], reverse = True) 
        
        qids, Q0s, docs, scores, tags = zip(*part_of_commom_list)
        
        ranks = range(1, len(scores) + 1)
        part_of_commom_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
        common_list = common_list + part_of_commom_list

    with open(output_dir + 'run_example1.txt', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))

In [ ]:
run_baseline()

1
[<DOM Element: topics at 0x7f327e9e1b90>]
50
lstm
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
rerequest Which is better, Canon or Nikon?
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
0
1
2
3
aa shape (1, 1, 64)
bb shape (1, 1, 64)
0
1
2
3


In [8]:
a = make_a_search_request("What is the difference between sex and love?")
a.json()

{'meta': {'query_time': 5724, 'total_results': 194416, 'indices': ['cw12']},
 'results': [{'score': 2406.7341,
   'uuid': 'b0054765-4007-52c9-a7c4-70c15891b4cc',
   'index': 'cw12',
   'trec_id': 'clueweb12-1214wb-88-29751',
   'target_hostname': 'articles.student.com',
   'target_uri': 'http://articles.student.com/health-sexuality/sexual-health/difference-between-love-and-sex',
   'page_rank': 1.1797776e-09,
   'spam_rank': 78,
   'title': '<em>Difference</em> <em>Between</em> <em>Love</em> <em>And</em> <em>Sex</em> - Student.com Articles',
   'snippet': '<em>Sex</em> may or may not include penetration. <em>Differences</em> <em>Between</em> <em>Love</em> <em>and</em> <em>Sex</em> <em>Love</em> <em>Love</em> <em>is</em> a feeling (emotional). There <em>is</em> no exact &quot;right&quot; definition of <em>love</em> for everybody. <em>Love</em> involves feelings of romance <em>and</em>&#x2F;or attraction. <em>Sex</em>: <em>Sex</em> <em>is</em> an event or act',
   'explanation': None},
 